# Lab 1b: Langchain Basics

In this lab, you will learn about LangChain — an open-source framework that helps developers build powerful applications using large language models (LLMs) like OpenAI's GPT. You'll start by installing the necessary packages, then explore chat models and basic tool usage, which will serve as a foundation for building AI agents in later labs.

## Installing LangChain
We'll start by installing the langchain package.

In [1]:
!pip install langchain
!pip install langchain-openai

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/ed/5c/5c0be747261e1f8129b875fa3bfea736bc5fe17652f9d5e15ca118571b6f/langchain-0.3.25-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<1.0.0,>=0.3.58 from https://files.pythonhosted.org/packages/c9/91/454a94275d323c0969e1f17a293cc87cb1398ab2d73f7db0a5de7883f2a9/langchain_core-0.3.58-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-splitters<1.0.0,>=0.3.8 from https://files.pythonhosted.org/packages/8b/a3/3696ff2444658053c01b6b7443e761f28bb71217d82bb89137a978c5f66f/langchain_text_splitters-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.4,>=0.1.17 from https://files.pythonhosted.org/packages/89/8e/e8a58e0abaae3f3ac4702e9ca35d1fc6159711556b64ffd0e247771a3f12/langsmith-0.3.42-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/

In [ ]:
# loading environment variables 
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

The most basic package in the Langchain ecosystem is langchain-core, which contains all classes and abstractions required to build other packages, except LangSmith package.  
<img src="https://python.langchain.com/assets/images/ecosystem_packages-32943b32657e7a187770c9b585f22a64.png" width="500">

## Learning about Messages
In Langchain, each message is defined by a role (e.g., "user", "assistant") and the content (e.g., text, multimodal data) with additional metadata that varies depending on the chat model provider. LangChain provides a unified message format that can be used across chat models, allowing users to work with different chat models without worrying about the specific details of the message format used by each model provider.

### Role 
Roles are used to distinguish between different types of messages in a conversation and help the chat model understand how to respond to a given sequence of messages.

| Role | Description |
|---|---|
|system|Used to tell the chat model how to behave and provide additional context. Not supported by all chat model providers.|
|user|Represents input from a user interacting with the model, usually in the form of text or other interactive input.|
|assistant|Represents a response from the model, which can include text or a request to invoke tools.|
|tool|A message used to pass the results of a tool invocation back to the model after external data or processing has been retrieved. Used with chat models that support tool calling.|

### Langchain Messages 
SystemMessage: corresponds to system role

HumanMessage: corresponds to user role

AIMessage: corresponds to assistant role

AIMessageChunk: corresponds to assistant role, used for streaming responses

ToolMessage: corresponds to tool role

RemoveMessage -- does not correspond to any role. This is an abstraction, mostly used in LangGraph to manage chat history.


In [41]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AIMessageChunk, RemoveMessage
import json 

human_message = HumanMessage("Hello, I am a human.")
system_message = SystemMessage("Hello, I am a system message.")
ai_message = AIMessage("Hello, I am an AI.")
ai_message_chunk = AIMessageChunk("I am a chunk of AI Message.")
remove_message = RemoveMessage(id="123")

print("Human Message")
print(human_message.model_dump_json(indent=2))

print("System Message")
print(system_message.model_dump_json(indent=2))

print("AI Message")
print(ai_message.model_dump_json(indent=2))

print("AI Message Chunk")
print(ai_message_chunk.model_dump_json(indent=2))

print("Removed Message")
print(remove_message.model_dump_json(indent=2))

Human Message
{
  "content": "Hello, I am a human.",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "human",
  "name": null,
  "id": null,
  "example": false
}
System Message
{
  "content": "Hello, I am a system message.",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "system",
  "name": null,
  "id": null
}
AI Message
{
  "content": "Hello, I am an AI.",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "ai",
  "name": null,
  "id": null,
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": null
}
AI Message Chunk
{
  "content": "I am a chunk of AI Message.",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "AIMessageChunk",
  "name": null,
  "id": null,
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": null,
  "tool_call_chunks": []
}
Removed Message
{
  "content": "",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "remove",
  "name

## Prompt Templates
Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

In [53]:
# Simple Prompt Template

from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Berapa total penjualan produk {name}")

prompt_template.invoke({"name":"A"})

StringPromptValue(text='Berapa total penjualan produk A')

In [65]:
# ChatPromptTemplates 
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate( [
    ("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
    ("user", "Berapa total penjualan produk {name}")
])

prompt_template.invoke({"name": "A"}).to_messages()

[SystemMessage(content='You are a database expert and your task is to write a SQL statement based on a question from user. The SQL query statement shall be executed against an sqlite database.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Berapa total penjualan produk A', additional_kwargs={}, response_metadata={})]

In [66]:
# MessagesPlaceholder 
# Inserting the whole message instead of keywords.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate(
    [("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
      MessagesPlaceholder("msg")
    ]
)

prompt_template.invoke({"msg": [SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH'), 
                                AIMessage(content="Berapa total penjualan produk A")]}).to_messages()

[SystemMessage(content='You are a database expert and your task is to write a SQL statement based on a question from user. The SQL query statement shall be executed against an sqlite database.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Berapa total penjualan produk A', additional_kwargs={}, response_metadata={})]

In [67]:
# Combine them

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate(
    [("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
      MessagesPlaceholder("msg"),
      ("user", "Berapa jumlah produk {name} yang terjual di bulan {month}")
    ]
)

prompt_template.invoke({"name":"A", 
                        "month":"3", 
                        "msg": [SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH')]
                        }).to_messages()

[SystemMessage(content='You are a database expert and your task is to write a SQL statement based on a question from user. The SQL query statement shall be executed against an sqlite database.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Berapa jumlah produk A yang terjual di bulan 3', additional_kwargs={}, response_metadata={})]

## Chat Models
Chat models are language models that use a sequence of messages as inputs and return messages as outputs (as opposed to using plain text). 

In [70]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1-mini", model_provider= "openai")
model.invoke("Please introduce yourself")

AIMessage(content="Hello! I'm ChatGPT, an AI language model created by OpenAI. I'm here to assist you with a wide range of topics, answer questions, provide explanations, help with writing, and much more. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 10, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-BTSggYhdsSj4Kjz2MckbkSlwgnTgX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--788464fb-46f0-49fb-8fc0-04520b63e1a2-0', usage_metadata={'input_tokens': 10, 'output_tokens': 49, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reaso

In [72]:
# How to stream chat model responses

for chunk in model.stream("Write me a 1 verse song about goldfish on the moon"):
    print(chunk.content, end="", flush=True)

Golden fins that shimmer bright,  
Gliding through the silver night,  
Moonlit waves in zero gloom,  
Dreaming deep—a goldfish moon.

## Output Parsers
Using the .with_structured_output() method 

In [84]:
# Pydantic Class 

from pydantic import BaseModel, Field

class ConceptList(BaseModel):
    """The list of concepts and brief description"""
    concept: str = Field(description="Important concept to be explained.")
    explanation: str = Field(description="Brief explanation of the concept")

structured_model = model.with_structured_output(ConceptList)

structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")

ConceptList(concept='Transformer Architecture', explanation='Transformer architecture is a neural network design that uses self-attention mechanisms to process input data, enabling efficient handling of sequences and is foundational in many generative AI models like GPT and BERT.')

If you don't want to use Pydantic, explicitly don't want validation of the arguments, or want to be able to stream the model outputs, you can define your schema using a TypedDict class. We can optionally use a special Annotated syntax supported by LangChain that allows you to specify the default value and description of a field. Note, the default value is not filled in automatically if the model doesn't generate it, it is only used in defining the schema that is passed to the model.

Requirements

> Core: langchain-core>=0.2.26

> Typing extensions: It is highly recommended to import Annotated and TypedDict from typing_extensions instead of typing to ensure consistent behavior across Python versions.

In [85]:
# TypedDict 

from typing_extensions import Annotated, TypedDict

class ConceptList(TypedDict):
    """The list of concepts and brief description"""
    concept: Annotated[str,..., "Important concept to be explained."] # type, default value, and description
    explanation: Annotated[str, ... , "Brief explanation of the concept"]

structured_model = model.with_structured_output(ConceptList)
structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")


{'concept': 'Transformer Architecture',
 'explanation': 'Transformer architecture is a neural network design that uses self-attention mechanisms to process sequences of data, enabling efficient handling of long-range dependencies. It forms the foundation for many generative AI models like GPT, allowing them to generate coherent and contextually relevant text.'}

In [89]:
# JSON Schema
# Equivalently, we can pass in a JSON Schema dict. 
# This requires no imports or classes and makes it very clear exactly how each parameter is documented, 
# at the cost of being a bit more verbose.

json_schema = {
    "title": "ConceptList",
    "description": "The list of concepts and brief description",
    "type": "object",
    "properties": {
        "concept": {
            "type": "string",
            "description": "Important concept to be explained."
        },
        "explanation": {
            "type": "string",
            "description": "Brief explanation of the concept."
        }
    },
    "required": ["concept", "explanation"]
}

structured_model = model.with_structured_output(json_schema)
structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")


{'concept': 'Transformer Architecture',
 'explanation': 'The Transformer architecture is a type of deep learning model that uses self-attention mechanisms to process and generate sequences of data, such as text. It enables generative AI models to understand context and produce coherent and contextually relevant outputs.'}

In [90]:
# Choosing Multiple Schema

from pydantic import BaseModel, Field
from typing import Union

class ConceptList(BaseModel):
    """The list of concepts and brief description"""
    concept: str = Field(description="Important concept to be explained.")
    explanation: str = Field(description="Brief explanation of the concept")

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[ConceptList, ConversationalResponse]


structured_model = model.with_structured_output(FinalResponse)

structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")

FinalResponse(final_output=ConceptList(concept='Transformer Architecture', explanation='Transformer architecture is a deep learning model architecture introduced in 2017. It uses self-attention mechanisms to process input data sequences, enabling models to understand context and relationships in data effectively. Transformers are foundational in many generative AI systems like GPT, allowing them to generate coherent and contextually relevant text.'))

In [91]:
structured_model.invoke("How do you feel about generative AI technology?")

FinalResponse(final_output=ConversationalResponse(response="As an AI, I don't have feelings or consciousness, but I can share that generative AI technology is a powerful tool with many beneficial applications, such as enhancing creativity, automating tasks, and assisting with problem-solving. However, like all technologies, it also raises important ethical considerations and challenges that society needs to address thoughtfully."))

## Prompt Chaining

In [95]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# Step 1: Set up the LLM
llm = init_chat_model("gpt-4.1-mini", model_provider= "openai", temperature = 0.7)
llm.invoke("Please introduce yourself")

AIMessage(content='Hello! I’m ChatGPT, an AI language model created by OpenAI. I’m here to help with answering questions, providing explanations, brainstorming ideas, writing assistance, and much more. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 10, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-BTVTmIVl9UQhQ2OUMow3bfhk9BPTu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d226c97c-e98e-42b1-8b1b-9a7e05812a6f-0', usage_metadata={'input_tokens': 10, 'output_tokens': 46, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

In [97]:
# Chain 1: Summarize input text
summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following article:\n\n{text}"
)

summarize_chain = summarize_prompt | llm

In [124]:
input_text = """
Artificial Intelligence (AI) is transforming industries by enabling machines to learn from data, 
make decisions, and even improve over time. Applications range from chatbots and virtual assistants 
to complex data analytics and autonomous vehicles. However, AI also brings challenges such as ethical concerns, 
bias in algorithms, and job displacement. As AI continues to evolve, balancing innovation with responsible 
development will be key to its long-term success.
"""

full_response = ""
for chunk in summarize_chain.stream(input=input_text):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

The article discusses how Artificial Intelligence (AI) is revolutionizing various industries by allowing machines to learn, make decisions, and improve over time. It highlights AI applications like chatbots, virtual assistants, data analytics, and autonomous vehicles. The article also addresses challenges including ethical issues, algorithmic bias, and job displacement. It emphasizes the importance of balancing innovation with responsible development for AI's sustainable success.

In [125]:
# Chain 2: Generate 3 quiz questions from the summary
question_prompt = PromptTemplate(
    input_variables=["summary"],
    template="Based on the summary below, generate 3 quiz questions:\n\n{summary}"
)
question_chain = question_prompt | llm

for chunk in question_chain.stream(input=full_response):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

1. What are some key applications of Artificial Intelligence mentioned in the article?  
2. What challenges related to AI development does the article highlight?  
3. Why does the article emphasize balancing innovation with responsible development in AI?

In [126]:
# Chain 3: Answer the first question
answer_prompt = PromptTemplate(
    input_variables=["questions"],
    template="Pick the first question from below and answer it in detail:\n\n{questions}"
)
answer_chain = answer_prompt | llm

for chunk in answer_chain.stream(input=full_response):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

The article mentions several key applications of Artificial Intelligence (AI) that are revolutionizing various industries:

1. **Chatbots:** These AI-powered programs simulate human conversation, enabling businesses to provide instant customer support, answer queries, and improve user engagement without the need for human intervention. Chatbots are widely used in e-commerce, customer service, and information dissemination.

2. **Virtual Assistants:** Examples include AI systems like Siri, Alexa, and Google Assistant, which help users perform tasks such as setting reminders, searching the internet, controlling smart home devices, and managing schedules through natural language processing and machine learning.

3. **Data Analytics:** AI enhances data analytics by processing vast amounts of data to identify patterns, trends, and insights that humans might miss. This capability aids decision-making, forecasting, and strategic planning in sectors like finance, healthcare, marketing, and man

In [118]:
# Step 2: Compose full chain
full_chain = summarize_chain | question_chain | answer_chain

for chunk in full_chain.stream(input=input_text):
     print(chunk.content, end="", flush=True)

Since the first question is: "What are some of the key applications of Artificial Intelligence mentioned in the article?" I will provide a detailed answer based on general knowledge, as the specific article content is not provided.

Artificial Intelligence (AI) has a broad range of applications across various industries. Some of the key applications typically highlighted include:

1. **Healthcare:** AI is used for diagnostics, personalized medicine, drug discovery, and robotic surgeries. Machine learning algorithms can analyze medical images to detect diseases like cancer more accurately and faster than traditional methods.

2. **Finance:** AI powers fraud detection, algorithmic trading, credit scoring, and personalized financial advice. It helps in analyzing vast amounts of financial data to detect anomalies and forecast market trends.

3. **Transportation:** AI enables autonomous vehicles, traffic management systems, and predictive maintenance of vehicles and infrastructure. Self-dri